## Reshaping Dataframes


In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa

### Concatenating pd.DataFrame objects

In [2]:
df_q1 = pd.DataFrame([
    ["AAPL", 100., 50., 75.],
    ["MSFT", 80., 42., 62.],
    ["AMZN", 60., 100., 120.],
], columns=["ticker", "shares", "low", "high"])
df_q1 = df_q1.convert_dtypes(dtype_backend="numpy_nullable")

df_q1

,ticker,shares,low,high
0,AAPL,100,50,75
1,MSFT,80,42,62
2,AMZN,60,100,120


In [3]:
df_q2 = pd.DataFrame([
    ["AAPL", 80., 70., 80., 77.],
    ["MSFT", 90., 50., 60., 55.],
    ["IBM", 100., 60., 70., 64.],
    ["GE", 42., 30., 50., 44.],
], columns=["ticker", "shares", "low", "high", "close"])
df_q2 = df_q2.convert_dtypes(dtype_backend="numpy_nullable")

df_q2

,ticker,shares,low,high,close
0,AAPL,80,70,80,77
1,MSFT,90,50,60,55
2,IBM,100,60,70,64
3,GE,42,30,50,44


In [4]:
pd.concat([df_q1, df_q2])

,ticker,shares,low,high,close
0,AAPL,100,50,75,<NA>
1,MSFT,80,42,62,<NA>
2,AMZN,60,100,120,<NA>
0,AAPL,80,70,80,77
1,MSFT,90,50,60,55
2,IBM,100,60,70,64
3,GE,42,30,50,44


In [5]:
pd.concat([df_q1, df_q2], ignore_index=True)

,ticker,shares,low,high,close
0,AAPL,100,50,75,<NA>
1,MSFT,80,42,62,<NA>
2,AMZN,60,100,120,<NA>
3,AAPL,80,70,80,77
4,MSFT,90,50,60,55
5,IBM,100,60,70,64
6,GE,42,30,50,44


In [6]:
pd.concat([df_q1, df_q2], keys=["q1", "q2"])

ticker  shares  low  high  close
q1 0   AAPL     100   50    75   <NA>
   1   MSFT      80   42    62   <NA>
   2   AMZN      60  100   120   <NA>
q2 0   AAPL      80   70    80     77
   1   MSFT      90   50    60     55
   2    IBM     100   60    70     64
   3     GE      42   30    50     44

In [7]:
pd.concat([df_q1, df_q2], keys=["q1", "q2"], axis=1)

q1                        q2                      
  ticker shares   low  high ticker shares low high close
0   AAPL    100    50    75   AAPL     80  70   80    77
1   MSFT     80    42    62   MSFT     90  50   60    55
2   AMZN     60   100   120    IBM    100  60   70    64
3   <NA>   <NA>  <NA>  <NA>     GE     42  30   50    44

In [8]:
pd.concat([
    df_q1.set_index("ticker"),
    df_q2.set_index("ticker"),
], keys=["q1", "q2"], axis=1)

q1                 q2                  
       shares   low  high shares   low  high close
ticker                                            
AAPL      100    50    75     80    70    80    77
MSFT       80    42    62     90    50    60    55
AMZN       60   100   120   <NA>  <NA>  <NA>  <NA>
IBM      <NA>  <NA>  <NA>    100    60    70    64
GE       <NA>  <NA>  <NA>     42    30    50    44

In [9]:
pd.concat([
    df_q1.set_index("ticker"),
    df_q2.set_index("ticker"),
], keys=["q1", "q2"], axis=1, join="inner")

q1              q2               
       shares low high shares low high close
ticker                                      
AAPL      100  50   75     80  70   80    77
MSFT       80  42   62     90  50   60    55

In [10]:
%%time
concatenated_dfs = df_q1
for i in range(1000):
    concatenated_dfs = pd.concat([concatenated_dfs, df_q1])

print(f"Final pd.DataFrame shape is {concatenated_dfs.shape}")

Final pd.DataFrame shape is (3003, 4)
CPU times: total: 484 ms
Wall time: 567 ms


In [11]:
%%time
df = df_q1
accumulated = [df_q1]
for i in range(1000):
    accumulated.append(df_q1)

concatenated_dfs = pd.concat(accumulated)
print(f"Final pd.DataFrame shape is {concatenated_dfs.shape}")

Final pd.DataFrame shape is (3003, 4)
CPU times: total: 93.8 ms
Wall time: 110 ms


### Merging dataframes with pd.merge

In [12]:
df_q1 = pd.DataFrame([
    ["AAPL", 100., 50., 75.],
    ["MSFT", 80., 42., 62.],
    ["AMZN", 60., 100., 120.],
], columns=["ticker", "shares", "low", "high"])
df_q1 = df_q1.convert_dtypes(dtype_backend="numpy_nullable")

df_q1

,ticker,shares,low,high
0,AAPL,100,50,75
1,MSFT,80,42,62
2,AMZN,60,100,120


In [13]:
df_q2 = pd.DataFrame([
    ["AAPL", 80., 70., 80., 77.],
    ["MSFT", 90., 50., 60., 55.],
    ["IBM", 100., 60., 70., 64.],
    ["GE", 42., 30., 50., 44.],
], columns=["ticker", "shares", "low", "high", "close"])
df_q2 = df_q2.convert_dtypes(dtype_backend="numpy_nullable")

df_q2

,ticker,shares,low,high,close
0,AAPL,80,70,80,77
1,MSFT,90,50,60,55
2,IBM,100,60,70,64
3,GE,42,30,50,44


In [14]:
pd.concat([
    df_q1.set_index("ticker"),
    df_q2.set_index("ticker"),
], keys=["q1", "q2"], axis=1)

q1                 q2                  
       shares   low  high shares   low  high close
ticker                                            
AAPL      100    50    75     80    70    80    77
MSFT       80    42    62     90    50    60    55
AMZN       60   100   120   <NA>  <NA>  <NA>  <NA>
IBM      <NA>  <NA>  <NA>    100    60    70    64
GE       <NA>  <NA>  <NA>     42    30    50    44

In [15]:
pd.merge(df_q1, df_q2, on=["ticker"])

,ticker,shares_x,low_x,high_x,shares_y,low_y,high_y,close
0,AAPL,100,50,75,80,70,80,77
1,MSFT,80,42,62,90,50,60,55


In [17]:
pd.merge(df_q1, df_q2, on=["ticker"], how="outer")  # how="right", "left", "inner"

,ticker,shares_x,low_x,high_x,shares_y,low_y,high_y,close
0,AAPL,100,50,75,80,70,80,77
1,AMZN,60,100,120,<NA>,<NA>,<NA>,<NA>
2,GE,<NA>,<NA>,<NA>,42,30,50,44
3,IBM,<NA>,<NA>,<NA>,100,60,70,64
4,MSFT,80,42,62,90,50,60,55


In [18]:
pd.merge(
    df_q1,
    df_q2,
    on=["ticker"],
    how="outer",
    suffixes=("_q1", "_q2"),
)

,ticker,shares_q1,low_q1,high_q1,shares_q2,low_q2,high_q2,close
0,AAPL,100,50,75,80,70,80,77
1,AMZN,60,100,120,<NA>,<NA>,<NA>,<NA>
2,GE,<NA>,<NA>,<NA>,42,30,50,44
3,IBM,<NA>,<NA>,<NA>,100,60,70,64
4,MSFT,80,42,62,90,50,60,55


In [19]:
pd.merge(
    df_q1[["ticker"]].assign(only_in_left=42),
    df_q2[["ticker"]].assign(only_in_right=555),
    on=["ticker"],
    how="outer",
    suffixes=("_q1", "_q2"),
)

,ticker,only_in_left,only_in_right
0,AAPL,42.0,555.0
1,AMZN,42.0,NaN
2,GE,NaN,555.0
3,IBM,NaN,555.0
4,MSFT,42.0,555.0


In [20]:
df_q2 = df_q2.rename(columns={"ticker": "SYMBOL"})

df_q2

,SYMBOL,shares,low,high,close
0,AAPL,80,70,80,77
1,MSFT,90,50,60,55
2,IBM,100,60,70,64
3,GE,42,30,50,44


In [21]:
pd.merge(
    df_q1,
    df_q2,
    left_on=["ticker"],
    right_on=["SYMBOL"],
    how="outer",
    suffixes=("_q1", "_q2"),
)

,ticker,shares_q1,low_q1,high_q1,SYMBOL,shares_q2,low_q2,high_q2,close
0,AAPL,100,50,75,AAPL,80,70,80,77
1,AMZN,60,100,120,<NA>,<NA>,<NA>,<NA>,<NA>
2,<NA>,<NA>,<NA>,<NA>,GE,42,30,50,44
3,<NA>,<NA>,<NA>,<NA>,IBM,100,60,70,64
4,MSFT,80,42,62,MSFT,90,50,60,55


In [22]:
lows = pd.DataFrame([
    ["AAPL", "Q1", 50.],
    ["MSFT", "Q1", 42.],
    ["AMZN", "Q1", 100.],
    ["AAPL", "Q2", 70.],
    ["MSFT", "Q2", 50.],
    ["IBM", "Q2", 60.],
    ["GE", "Q2", 30.],
], columns=["ticker", "quarter", "low"])
lows = lows.convert_dtypes(dtype_backend="numpy_nullable")

lows

,ticker,quarter,low
0,AAPL,Q1,50
1,MSFT,Q1,42
2,AMZN,Q1,100
3,AAPL,Q2,70
4,MSFT,Q2,50
5,IBM,Q2,60
6,GE,Q2,30


In [23]:
highs = pd.DataFrame([
    ["AAPL", "Q1", 75.],
    ["MSFT", "Q1", 62.],
    ["AMZN", "Q1", 120.],
    ["AAPL", "Q2", 80.],
    ["MSFT", "Q2", 60.],
    ["IBM", "Q2", 70.],
    ["GE", "Q2", 50.],
], columns=["SYMBOL", "QTR", "high"])
highs = highs.convert_dtypes(dtype_backend="numpy_nullable")

highs

,SYMBOL,QTR,high
0,AAPL,Q1,75
1,MSFT,Q1,62
2,AMZN,Q1,120
3,AAPL,Q2,80
4,MSFT,Q2,60
5,IBM,Q2,70
6,GE,Q2,50


In [24]:
pd.merge(
    lows,
    highs,
    left_on=["ticker", "quarter"],
    right_on=["SYMBOL", "QTR"],
)

,ticker,quarter,low,SYMBOL,QTR,high
0,AAPL,Q1,50,AAPL,Q1,75
1,MSFT,Q1,42,MSFT,Q1,62
2,AMZN,Q1,100,AMZN,Q1,120
3,AAPL,Q2,70,AAPL,Q2,80
4,MSFT,Q2,50,MSFT,Q2,60
5,IBM,Q2,60,IBM,Q2,70
6,GE,Q2,30,GE,Q2,50


### Joining dataframes with pd.DataFrame.join

In [25]:
sales = pd.DataFrame(
    [[1000], [2000], [4000]],
    columns=["sales"],
    index=pd.Index([42, 555, 9000], name="salesperson_id")
)
sales = sales.convert_dtypes(dtype_backend="numpy_nullable")
sales

,sales
salesperson_id,
42,1000
555,2000
9000,4000


In [26]:
salesperson = pd.DataFrame([
    ["John", "Smith"],
    ["Jane", "Doe"],
], columns=["first_name", "last_name"], index=pd.Index(
    [555, 42], name="salesperson_id"
))
salesperson = salesperson.convert_dtypes(dtype_backend="numpy_nullable")
salesperson

,first_name,last_name
salesperson_id,,
555,John,Smith
42,Jane,Doe


In [27]:
pd.merge(sales, salesperson, left_index=True, right_index=True, how="left")

,sales,first_name,last_name
salesperson_id,,,
42,1000,Jane,Doe
555,2000,John,Smith
9000,4000,<NA>,<NA>


In [28]:
sales.join(salesperson)

,sales,first_name,last_name
salesperson_id,,,
42,1000,Jane,Doe
555,2000,John,Smith
9000,4000,<NA>,<NA>


In [29]:
sales.join(salesperson, how="inner")

,sales,first_name,last_name
salesperson_id,,,
42,1000,Jane,Doe
555,2000,John,Smith


### Reshaping with pd.DataFrame.stack and pd.DataFrame.unstack

In [30]:
df = pd.DataFrame([
    [12, 10, 40],
    [9, 7, 12],
    [0, 14, 190]
], columns=pd.Index(["Apple", "Orange", "Banana"], name="fruit"), index=pd.Index(
    ["Texas", "Arizona", "Florida"], name="state"))

df = df.convert_dtypes(dtype_backend="numpy_nullable")

df

fruit,Apple,Orange,Banana
state,,,
Texas,12,10,40
Arizona,9,7,12
Florida,0,14,190


In [31]:
df.stack()

state    fruit 
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: Int64

In [32]:
df.stack().reset_index(name="number_grown")

,state,fruit,number_grown
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [33]:
stacked = df.stack()
stacked

state    fruit 
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: Int64

In [34]:
stacked.unstack(level=0)

state,Texas,Arizona,Florida
fruit,,,
Apple,12,9,0
Orange,10,7,14
Banana,40,12,190


In [35]:
stacked.unstack(level="state")

state,Texas,Arizona,Florida
fruit,,,
Apple,12,9,0
Orange,10,7,14
Banana,40,12,190


### Reshaping with pd.DataFrame.melt

In [36]:
df = pd.DataFrame([
    ["Texas", 12, 10, 40],
    ["Arizona", 9, 7, 12],
    ["Florida", 0, 14, 190]
], columns=["state", "apple", "orange", "banana"])
df = df.convert_dtypes(dtype_backend="numpy_nullable")

df

,state,apple,orange,banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [37]:
df.set_index("state").stack().reset_index()

,state,level_1,0
0,Texas,apple,12
1,Texas,orange,10
2,Texas,banana,40
3,Arizona,apple,9
4,Arizona,orange,7
5,Arizona,banana,12
6,Florida,apple,0
7,Florida,orange,14
8,Florida,banana,190


In [38]:
df.set_index("state").stack().reset_index().rename(columns={
    "level_1": "fruit",
    0: "number_grown",
})

,state,fruit,number_grown
0,Texas,apple,12
1,Texas,orange,10
2,Texas,banana,40
3,Arizona,apple,9
4,Arizona,orange,7
5,Arizona,banana,12
6,Florida,apple,0
7,Florida,orange,14
8,Florida,banana,190


In [39]:
df.melt(id_vars=["state"])

,state,variable,value
0,Texas,apple,12
1,Arizona,apple,9
2,Florida,apple,0
3,Texas,orange,10
4,Arizona,orange,7
5,Florida,orange,14
6,Texas,banana,40
7,Arizona,banana,12
8,Florida,banana,190


### Reshaping with pd.wide_to_long

In [40]:
df = pd.DataFrame([
    ["Widget 1", 1, 2, 4, 8],
    ["Widget 2", 16, 32, 64, 128],
], columns=["widget", "quarter_1", "quarter_2", "quarter_3", "quarter_4"])
df = df.convert_dtypes(dtype_backend="numpy_nullable")

df

,widget,quarter_1,quarter_2,quarter_3,quarter_4
0,Widget 1,1,2,4,8
1,Widget 2,16,32,64,128


In [41]:
df.set_index("widget").stack().reset_index().rename(columns={
    "level_1": "quarter",
    0: "quantity",
})

,widget,quarter,quantity
0,Widget 1,quarter_1,1
1,Widget 1,quarter_2,2
2,Widget 1,quarter_3,4
3,Widget 1,quarter_4,8
4,Widget 2,quarter_1,16
5,Widget 2,quarter_2,32
6,Widget 2,quarter_3,64
7,Widget 2,quarter_4,128


In [42]:
df.melt(
    id_vars=["widget"],
    var_name="quarter",
    value_name="quantity",
)

,widget,quarter,quantity
0,Widget 1,quarter_1,1
1,Widget 2,quarter_1,16
2,Widget 1,quarter_2,2
3,Widget 2,quarter_2,32
4,Widget 1,quarter_3,4
5,Widget 2,quarter_3,64
6,Widget 1,quarter_4,8
7,Widget 2,quarter_4,128


In [43]:
pd.wide_to_long(
    df,
    i=["widget"],
    stubnames="quarter_",
    j="quarter"
).reset_index().rename(columns={"quarter_": "quantity"})

,widget,quarter,quantity
0,Widget 1,1,1
1,Widget 2,1,16
2,Widget 1,2,2
3,Widget 2,2,32
4,Widget 1,3,4
5,Widget 2,3,64
6,Widget 1,4,8
7,Widget 2,4,128


### Reshaping with pd.DataFrame.pivot and pd.pivot_table

In [44]:
df = pd.DataFrame([
    ["Texas", "apple", 12, 8],
    ["Arizona", "apple", 9, 10],
    ["Florida", "apple", 0, 6],
    ["Texas", "orange", 10, 4],
    ["Arizona", "orange", 7, 2],
    ["Florida", "orange", 14, 3],
    ["Texas", "banana", 40, 28],
    ["Arizona", "banana", 12, 17],
    ["Florida", "banana", 190, 42],
], columns=["state", "fruit", "number_grown", "number_eaten"])
df = df.convert_dtypes(dtype_backend="numpy_nullable")

df

,state,fruit,number_grown,number_eaten
0,Texas,apple,12,8
1,Arizona,apple,9,10
2,Florida,apple,0,6
3,Texas,orange,10,4
4,Arizona,orange,7,2
5,Florida,orange,14,3
6,Texas,banana,40,28
7,Arizona,banana,12,17
8,Florida,banana,190,42


In [45]:
df.set_index(["state", "fruit"]).unstack()

number_grown               number_eaten              
fruit          apple banana orange        apple banana orange
state                                                        
Arizona            9     12      7           10     17      2
Florida            0    190     14            6     42      3
Texas             12     40     10            8     28      4

In [46]:
df.pivot(index=["state"], columns=["fruit"])

number_grown               number_eaten              
fruit          apple banana orange        apple banana orange
state                                                        
Arizona            9     12      7           10     17      2
Florida            0    190     14            6     42      3
Texas             12     40     10            8     28      4

In [47]:
df.pivot(
      index=["state"],
      columns=["fruit"],
      values=["number_grown"],
  )

number_grown              
fruit          apple banana orange
state                             
Arizona            9     12      7
Florida            0    190     14
Texas             12     40     10

In [48]:
wide_df = df.pivot(
    index=["state"],
    columns=["fruit"],
    values=["number_grown"],
).droplevel(level=0, axis=1)

wide_df

fruit,apple,banana,orange
state,,,
Arizona,9,12,7
Florida,0,190,14
Texas,12,40,10


In [49]:
df = pd.DataFrame([
    ["Texas", "apple", 2023, 10, 6],
    ["Texas", "apple", 2024, 2, 8],
    ["Arizona", "apple", 2023, 3, 7],
    ["Arizona", "apple", 2024, 6, 3],
    ["Texas", "orange", 2023, 5, 2],
    ["Texas", "orange", 2024, 5, 2],
    ["Arizona", "orange", 2023, 7, 2],
], columns=["state", "fruit", "year", "number_grown", "number_eaten"])
df = df.convert_dtypes(dtype_backend="numpy_nullable")

df

,state,fruit,year,number_grown,number_eaten
0,Texas,apple,2023,10,6
1,Texas,apple,2024,2,8
2,Arizona,apple,2023,3,7
3,Arizona,apple,2024,6,3
4,Texas,orange,2023,5,2
5,Texas,orange,2024,5,2
6,Arizona,orange,2023,7,2


In [50]:
df.pivot(
    index=["state", "year"],
    columns=["fruit"],
    values=["number_grown", "number_eaten"]
)

number_grown        number_eaten       
fruit               apple orange        apple orange
state   year                                        
Arizona 2023            3      7            7      2
        2024            6    NaN            3    NaN
Texas   2023           10      5            6      2
        2024            2      5            8      2

In [51]:
pd.pivot_table(
    df,
    index=["state"],
    columns=["fruit"],
    values=["number_grown", "number_eaten"]
)

number_eaten        number_grown       
fruit          apple orange        apple orange
state                                          
Arizona          5.0    2.0          4.5    7.0
Texas            7.0    2.0          6.0    5.0

In [52]:
pd.pivot_table(
    df,
    index=["state"],
    columns=["fruit"],
    values=["number_grown", "number_eaten"],
    aggfunc="sum"
)

number_eaten        number_grown       
fruit          apple orange        apple orange
state                                          
Arizona           10      2            9      7
Texas             14      4           12     10

In [53]:
pd.pivot_table(
    df,
    index=["state"],
    columns=["fruit"],
    values=["number_grown", "number_eaten"],
    aggfunc={
        "number_eaten": ["min", "max"],
        "number_grown": ["sum", "mean"],
    },
)

number_eaten                     number_grown                    
                 max          min                mean          sum       
fruit          apple orange apple orange        apple orange apple orange
state                                                                    
Arizona            7      2     3      2          4.5    7.0     9      7
Texas              8      2     6      2          6.0    5.0    12     10

### Reshaping with pd.DataFrame.explode

In [54]:
df = pd.DataFrame(
    [
        {
            "employee_id": 1,
            "first_name": "John",
            "last_name": "Smith",
            "direct_reports": [2, 3]
        },
        {
            "employee_id": 2,
            "first_name": "Jane",
            "last_name": "Doe",
            "direct_reports": []
        },
        {
            "employee_id": 3,
            "first_name": "Joe",
            "last_name": "Schmoe",
            "direct_reports": []
        }
    ]
)
df = df.convert_dtypes(dtype_backend="numpy_nullable")

df

,employee_id,first_name,last_name,direct_reports
0,1,John,Smith,"[2, 3]"
1,2,Jane,Doe,[]
2,3,Joe,Schmoe,[]


In [55]:
df.explode("direct_reports").convert_dtypes(dtype_backend="numpy_nullable")

,employee_id,first_name,last_name,direct_reports
0,1,John,Smith,2
0,1,John,Smith,3
1,2,Jane,Doe,<NA>
2,3,Joe,Schmoe,<NA>


In [56]:
exploded = df.explode("direct_reports").convert_dtypes(
    dtype_backend="numpy_nullable"
)
pd.merge(
    exploded,
    df.drop(columns=["direct_reports"]),
    how="left",
    left_on=["direct_reports"],
    right_on=["employee_id"],
    suffixes=("", "_direct_report"),
)

,employee_id,first_name,last_name,direct_reports,employee_id_direct_report,first_name_direct_report,last_name_direct_report
0,1,John,Smith,2,2,Jane,Doe
1,1,John,Smith,3,3,Joe,Schmoe
2,2,Jane,Doe,<NA>,<NA>,<NA>,<NA>
3,3,Joe,Schmoe,<NA>,<NA>,<NA>,<NA>


### Transposing with pd.DataFrame.T

In [57]:
df = pd.DataFrame([
    [1, 2, 3],
    [4, 5, 6],
], columns=list("xyz"), index=list("ab"))

df

,x,y,z
a,1,2,3
b,4,5,6


In [58]:
df.T

,a,b
x,1,4
y,2,5
z,3,6


In [59]:
df.sum()

x    5
y    7
z    9
dtype: int64

In [60]:
df.sum(axis=1)

a     6
b    15
dtype: int64

In [61]:
np.random.seed(42)
df = pd.DataFrame(
    np.random.randint(10, size=(2, 10_000)),
    index=list("ab"),
)

df

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
a,6,3,7,4,6,9,2,6,7,4,...,7,3,6,3,4,6,9,2,9,4
b,2,4,2,9,6,0,8,8,4,8,...,4,2,9,5,1,5,5,1,5,5


In [62]:
df.sum(axis=1)

a    44972
b    45097
dtype: int64

In [63]:
df.T.sum()

a    44972
b    45097
dtype: int64

In [66]:
import timeit

def baseline_sum():
    for _ in range(100):
        df.sum(axis=1)

timeit.timeit(baseline_sum, number=100)

16.02708739999798

In [65]:
def transposed_sum():
    transposed = df.T
    for _ in range(100):
        transposed.sum()

timeit.timeit(transposed_sum, number=100)

2.9504862999892794